# Sistema de Procesamiento de Datos de Ventas

Este notebook procesa datos de ventas desde un archivo CSV, los almacena en SQLite, realiza limpieza de datos y genera resúmenes mensuales por región.

## Características
- Carga de datos desde CSV a SQLite
- Eliminación de duplicados y registros nulos
- Cálculo de resumen de ventas por región y mes
- Exportación a CSV del resumen procesado

---

## Paso 1: Clonar repositorio e instalar dependencias

In [ ]:
import os

repo_path = '/content/Prueba-Tecnica-Anepsa'

if not os.path.exists(repo_path):
    print("Clonando repositorio...")
    %cd /content
    !git clone https://github.com/Thomassemino/Prueba-Tecnica-Anepsa.git
    
    print("Reinstalando NumPy y Pandas para asegurar compatibilidad...")
    !pip uninstall -y numpy pandas
    !pip install numpy pandas
    
    print("Instalando dependencias adicionales...")
    !pip install -q python-dotenv schedule
    
    print("Instalación completa. Reiniciando runtime...")
    print("IMPORTANTE: Después del reinicio, ejecuta esta celda nuevamente.")
    
    os.kill(os.getpid(), 9)
else:
    %cd /content/Prueba-Tecnica-Anepsa
    print("Repositorio listo. Continúa con el siguiente paso.")

## Paso 2: Verificar datos de entrada

In [ ]:
import pandas as pd
from pathlib import Path

archivo_csv = "Archivo ventas_simuladas.csv"
df = pd.read_csv(archivo_csv)

print(f"Total de registros en el CSV: {len(df)}")
print(f"\nPrimeras 10 filas:")
display(df.head(10))

print(f"\nInformación del dataset:")
print(df.info())

print(f"\nEstadísticas descriptivas:")
display(df.describe())

## Paso 3: Ejecutar procesamiento de datos

In [ ]:
!python procesar_ventas.py

## Paso 4: Verificar resultados

In [ ]:
archivo_resumen = "resumen_ventas_mensual.csv"
df_resumen = pd.read_csv(archivo_resumen)

print(f"Total de registros en el resumen: {len(df_resumen)}")
print(f"\nResumen completo:")
display(df_resumen)

print(f"\nTotal de ventas por región:")
ventas_por_region = df_resumen.groupby('region')['total_ventas'].sum().sort_values(ascending=False)
display(ventas_por_region)

## Paso 5: Consultar base de datos SQLite directamente

In [ ]:
import sqlite3

conexion = sqlite3.connect('anepsa_ventas.db')

print("Consulta SQL: Top 10 ventas individuales más grandes\n")
query_top_ventas = """
SELECT 
    fecha,
    region,
    producto,
    cantidad,
    precio_unitario,
    (cantidad * precio_unitario) as venta_total
FROM ventas_raw
ORDER BY venta_total DESC
LIMIT 10
"""
df_top_ventas = pd.read_sql_query(query_top_ventas, conexion)
display(df_top_ventas)

print("\nConsulta SQL: Cantidad de ventas por producto\n")
query_productos = """
SELECT 
    producto,
    COUNT(*) as cantidad_transacciones,
    SUM(cantidad) as unidades_vendidas,
    SUM(cantidad * precio_unitario) as total_ventas
FROM ventas_raw
GROUP BY producto
ORDER BY total_ventas DESC
"""
df_productos = pd.read_sql_query(query_productos, conexion)
display(df_productos)

conexion.close()
print("\nAnálisis completado exitosamente")

## Paso 6: Descargar archivos generados

Puedes descargar el resumen generado y la base de datos SQLite.

In [ ]:
from google.colab import files

print("Descargando archivos generados...")
files.download('resumen_ventas_mensual.csv')
files.download('anepsa_ventas.db')
print("Archivos descargados exitosamente")